# OneHotEncoderEstimator tutorial in PySpark

In [47]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

- 두 개의 변수를 갖는 샘플 데이터프레임을 생성한다.
- category1 변수는 categorical 변수이고, category2 변수는 numeric 변수이다.

In [48]:
df = spark.createDataFrame([(0, "a", 1),
                            (1, "b", 2),
                            (2, "c", 3),
                            (3, "a", 4),
                            (4, "a", 4),
                            (5, "c", 3)],
                           ["id", "category1", "category2"])
df.show()

+---+---------+---------+
| id|category1|category2|
+---+---------+---------+
|  0|        a|        1|
|  1|        b|        2|
|  2|        c|        3|
|  3|        a|        4|
|  4|        a|        4|
|  5|        c|        3|
+---+---------+---------+



- 먼저, `OneHotEncoderEstimator`에 넣는 input은 모두 numeric이어야 한다. 따라서 값들이 string인 categorical 변수들은 모두 numeric으로 변환해야 한다.
- 이때 `StringIndexer`를 사용한다. 따라서 아래의 코드를 보면, categorical 변수인 category1만 `StringIndexer`으로 변환해주었다.
- `StringIndexer`의 기능
    - categorical 변수를 numeric으로 변환한다.
    - categorical 변수의 값들의 빈도를 계산한다. 최빈값의 Index가 가장 낮다.

In [49]:
indexer = StringIndexer(inputCol="category1", outputCol="category1Index") # StringIndexer를 통해 categorical 변수의 categoryIndex를 계산한다.
inputs = [indexer.getOutputCol(), "category2"] # 위에서 설정한 indexer의 outputCol인 category1Index를 가져와서 numeric 변수인 category2와 함께 리스트로 만든다.
encoder = OneHotEncoderEstimator(inputCols=inputs, outputCols=["categoryVec1", "categoryVec2"])
pipeline = Pipeline(stages=[indexer, encoder])
pipeline.fit(df).transform(df).show()

+---+---------+---------+--------------+-------------+-------------+
| id|category1|category2|category1Index| categoryVec1| categoryVec2|
+---+---------+---------+--------------+-------------+-------------+
|  0|        a|        1|           0.0|(2,[0],[1.0])|(4,[1],[1.0])|
|  1|        b|        2|           2.0|    (2,[],[])|(4,[2],[1.0])|
|  2|        c|        3|           1.0|(2,[1],[1.0])|(4,[3],[1.0])|
|  3|        a|        4|           0.0|(2,[0],[1.0])|    (4,[],[])|
|  4|        a|        4|           0.0|(2,[0],[1.0])|    (4,[],[])|
|  5|        c|        3|           1.0|(2,[1],[1.0])|(4,[3],[1.0])|
+---+---------+---------+--------------+-------------+-------------+



- categorical 변수는 K-1개의 더미변수가 생성되고 numeric 변수는 K개의 더미변수가 생성된다.
- categorical 변수는 빈도가 가장 낮은 레벨이 베이스라인이 되지만, numeric 변수는 빈도가 가장 높은 레벨이 베이스라인이 된다.

In [50]:
a = pipeline.fit(df).transform(df)
a.toPandas()

,id,category1,category2,category1Index,categoryVec1,categoryVec2
0,0,a,1,0.0,"(1.0, 0.0)","(0.0, 1.0, 0.0, 0.0)"
1,1,b,2,2.0,"(0.0, 0.0)","(0.0, 0.0, 1.0, 0.0)"
2,2,c,3,1.0,"(0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)"
3,3,a,4,0.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 0.0)"
4,4,a,4,0.0,"(1.0, 0.0)","(0.0, 0.0, 0.0, 0.0)"
5,5,c,3,1.0,"(0.0, 1.0)","(0.0, 0.0, 0.0, 1.0)"
